# Book Recommendation System

## Part II Deep Dive of the project for E4571 Personalization: Theory & Application



| Group Member  | Name            | UNI    |
| ------------- |:-------------:  | -----: |
| 1             | Deepak Maran    | dm3308 |
| 2             |Rakshita Nagalla | rn2439 |
| 3             | Kewei Liu       | kl2987 |
| 4             |  Xiaohui Guo    | xg2225 |

---

## Introduction

Book crossing is a website that enables free sharing of books, either with friends or with complete strangers by wild-releasing books in public places. They offer optional paid membership for new members that includes features like book recommendation. These recommendations direct to the Better World Books website which is an online book-selling portal.

In this context, better top-k recommendations generate revenue to Bookcrossing in two ways:
    
    1.Better recommendations translate to purchases at World Books and Bookcrossings gets a share.
    2.Personalized recommendations leads to user satisfaction resulting in retention of users and brand value. This contributes to revenue through paid memberships as new users would opt for the optional membership and existing users would continue to pay to retain the membership.

Thus, our business objective is to generate k recommendations(top-k recommendation) for users logged into bookcrossings website.


## Challenge

We built a top-k recommendation engine for this task and explored neighborhood and model based collaborative filtering models for this task using collaborative filtering techniques in Part A. However, these methods are not able to recommend latest books to users as they have not been reviewed enough times. But recommending new books in the market that are relevant to the users is integral to driving sales for that book, and in turn for bookcrossing to generate revenue. In this project, we hope to design  recommender system that addresses this book cold start problem. 

Another important business objective is to generate personalized recommendations to new users to encourage them to opt for paid membership.  However, the collaborative filtering models we explored in Part A do not perform very well on new users who do not have enough rating history yet.

Thus our objective is to generate recommendations that work reasonably well even for new members and latest books. 


## Data

Density in percent: 0.0276993713304
Users: 5471 items: 12182

In addition to the ratings data we used in part-A, we leverage the book title, author and publisher information provided by book-crossings to deliver better recommendation and to address the cold-start problem.


## Models

We design a hybrid recommender system that generates recommendations to users. We use multiple hybridization strategies to combine predictions from a subset of the following three models:

   #### 1. kNN
   When implemented the k-nearest neighbor method, we found out that taking item-based similarity is seen to perform better than user-based similarity. The search for users in a large user population is the bottleneck in user-based methods. This is overcome by instead exploring relationships between items first which are smaller in number. A basic item-based kNN model with the Pearson similarity is seen to work best for our dataset.The basic kNN model was found to work best with respect to the evaluation metrics we want to optimize for. 

#### 2. SVD Matrix Facotorizaton
Due to SVD’s simplicity and effectiveness when using explicit data, we choose SVD matrix factorization as one of our models. The SVD model will learn the bias, such as certain users might not be very critical and tend to rate all book highly, or some users tend to rate all book very low. Also consider about the global bias term the follow formula shows the prediction used by SVD model. 





** insert formula here


 
![alt text](https://github.com/adam-p/markdown-here/raw/master/src/common/images/icon48.png "Logo Title Text 1")







The bias bu and the factors pu are assumed to be zero when user u is known. Similarly. When an item is known, the bias bi and the factor qi are assumed to be zero. The rating matrix is very sparse, which is with a density of 0.841%. We overcome this by adding regularization terms of the objective function. Here the goal is to minimize the following regularized regularized squared error.

#### 3. Content based models

    1). Item profile creation
    Once an item profile is created for each of the items in the dataset using the feature engineering methods described above, we estimate the rating user $u$ gave to item $i$ ($r_ui$) using a set ($I$) of $k$ items that are most similar to item $i$ among all the items user $u$ has rated.  Cosine distance between the two item profiles was used as proxy for similarity. Thus, the rating for user $u$ and item  $i$  is estimated as:
    
    
    $r_{ui} = b_u + \sum_{j \in I}sim(i,j)*r’_{ju}/ \sum_{j \in I}sim(i,j)$
    
   
insert formula here
 
![alt text](https://github.com/adam-p/markdown-here/raw/master/src/common/images/icon48.png "Logo Title Text 1")

    In order to account for the bias in the users’ ratings, r’_ui is calculated by subtracting the average rating of user $b_u$ from each of his ratings. 

    2). Feature engineering
    
        I. Publisher
        
        There are 16809 unique publishers in the dataset. Since it is a nominal variable, we only consider publishers who have published more than 5 books and one-hot encode them. The reasoning behind this is that most of the book are published by a select few publishers, hence ignoring publishers who published less than 5 book reducing the dimensionality greatly without losing much information. 



 ![alt text](./img/Publisher_ISBN.png "Number of ISBN per Publisher")
     

        II. Author
        
        There are 102,024 unique authors in the dataset. Since it is a nominal variable, we only consider authors who have written more than 5 books and one-hot encode them. The reasoning is same as that of encoding the publisher.

 ![alt text](./img/Author_books.png "Number of Books per Author")

        III. Title
        
        Before extracting features from the title of the book, we remove numbers, punctuations, stopwords and non-ascii characters. Further we perform lemmatization instead of stemming to reduce the word to root word, since we want the root word to retain its meaning to match with the word-embeddings. In addition, all the words are converted to lower-case alphabets.

              a.) Word embeddings:
              
               Since we only have information about the book titles, the corpus might not be sufficient to capture the meaning of words using just the bag of words model or tf-idf features. Hence, we used the word2vec model pre-trained on Common Crawl (CC) dataset. Word2vec model is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. Given enough data, usage and contexts, Word2vec can make highly accurate guesses about a word’s meaning based on past appearances. We take the aggregate of feature vectors for each of the words in the document in order to obtain a 300-dimensional feature vector for each of the titles. Out of the 11309 words in our corpus, word vectors were obtained for 10413 words after cleaning. These feature engineering techniques result in a feature vector of length 1026 for each book.
               
              Next, we explore the following hybridization strategies to combine these base models:
              
              



| Strategy No.  | Hybrid Strategy Name | 
| ------------- |:-------------:       | 
| 1             |  Switching Method    |  
| 2             |Weighted Method       |  
| 3             | Mix Method           |